In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None) 

In [2]:
file_path = "TushyCartonisationAnalysis_recreate.xlsx"
excel_sheets = pd.ExcelFile(file_path).sheet_names

In [3]:
invoice_df = pd.read_excel(file_path, sheet_name='Raw Baja Invoices')
invoice_df['id'] = range(1, len(invoice_df) + 1)
invoice_df['Charge'] = invoice_df['Charge'].str.upper()

In [4]:
charge_df = pd.read_excel(file_path, sheet_name='Charge Map')
charge_df['Charge'] = charge_df['Charge'].str.upper()
charge_map_df = charge_df[["Charge","Map"]]

In [5]:
invoice_charge_df = invoice_df.merge(charge_map_df, on='Charge', how='inner')

In [6]:
#missing Charge "Return To Sender - Web Request" and map "Returns"
invoice_charge_df["Map"].unique()

array(['Freight', 'Fuel', 'Residential', 'Surcharge', 'Freight Adj',
       'Fuel Adj', 'Surcharge Adj', 'Duties/Tax'], dtype=object)

In [ ]:
invoice_Flatten = invoice_charge_df.copy()
unique_maps = invoice_Flatten['Map'].unique()
for map_value in unique_maps:
    invoice_Flatten[map_value] = invoice_Flatten.apply(
        lambda row: row['Bill Amount'] if row['Map'] == map_value else 0, axis=1
    )

In [8]:
invoice_Flatten.head()

,Invoice,Invoice Date,Order ID,Order Number,Profile,Reference,Carrier,Transaction Date,Tracking Number,Service Type,Charge,Package Quantity,Packaging Type,Entered Weight (LB),Billed Weight (LB),Dim Length,Dim Width,Dim Height,Zone,Origin State,Origin Country,Receiver Name,Receiver Company,Receiver Address Line 1,Receiver Address Line 2,Receiver City,Receiver State,Receiver Zip,Receiver Country,Entry Date,Entry Number,Customs Value,Customs Value Currency,Duty & Taxes,Bill Amount,id,Map,Freight,Fuel,Residential,Surcharge,Freight Adj,Fuel Adj,Surcharge Adj,Duties/Tax
0,33477,2025-03-20,576899052,#1442861,default,NaN,UPS,45699,1ZAC8375YW34252402,UPS SurePost - 1 lb or Greater,TRANSPORTATION,1.0,PKG,4.1,5.0,19.0,9.0,5.0,7,TX,US,Dana Wallace,"TUSHY, Inc",10813 NW 30TH ST STE 115,NaN,DORAL,FL,331922147,US,NaN,NaN,0.0,NaN,0.0,9.80,1,Freight,9.80,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,33477,2025-03-20,576899052,#1442861,default,NaN,UPS,45699,1ZAC8375YW34252402,UPS SurePost - 1 lb or Greater,FUEL SURCHARGE,1.0,PKG,4.1,5.0,19.0,9.0,5.0,7,TX,US,Dana Wallace,"TUSHY, Inc",10813 NW 30TH ST STE 115,NaN,DORAL,FL,331922147,US,NaN,NaN,0.0,NaN,0.0,1.23,2,Fuel,0.00,1.23,0.0,0.0,0.0,0.0,0.0,0.0
2,33477,2025-03-20,581683521,#1446209,default,NaN,UPS,45712,1ZAC8375YW14943702,UPS SurePost - 1 lb or Greater,TRANSPORTATION,1.0,PKG,2.1,3.0,18.0,10.0,4.0,7,TX,US,NaN,Jacob Vosper,22 DORLAND AVE,NaN,POUGHKEEPSIE,NY,126036404,US,NaN,NaN,0.0,NaN,0.0,8.90,3,Freight,8.90,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,33477,2025-03-20,581683521,#1446209,default,NaN,UPS,45712,1ZAC8375YW14943702,UPS SurePost - 1 lb or Greater,FUEL SURCHARGE,1.0,PKG,2.1,3.0,18.0,10.0,4.0,7,TX,US,NaN,Jacob Vosper,22 DORLAND AVE,NaN,POUGHKEEPSIE,NY,126036404,US,NaN,NaN,0.0,NaN,0.0,1.12,4,Fuel,0.00,1.12,0.0,0.0,0.0,0.0,0.0,0.0
4,33477,2025-03-20,582604555,#1446902,default,NaN,UPS,45714,1ZAC83750301909861,Ground Residential,TRANSPORTATION,1.0,PKG,0.1,18.0,23.0,21.0,8.0,7,TX,US,NaN,Alisha Sare,277 SANTA ROSA AVE,PUSH 3 ON THE HILAVATOR OR TAKE,SAUSALITO,CA,949652036,US,NaN,NaN,0.0,NaN,0.0,16.86,5,Freight,16.86,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#4623466
invoice_Flatten[invoice_Flatten["Order Number"] == 4623466]

,Invoice,Invoice Date,Order ID,Order Number,Profile,Reference,Carrier,Transaction Date,Tracking Number,Service Type,Charge,Package Quantity,Packaging Type,Entered Weight (LB),Billed Weight (LB),Dim Length,Dim Width,Dim Height,Zone,Origin State,Origin Country,Receiver Name,Receiver Company,Receiver Address Line 1,Receiver Address Line 2,Receiver City,Receiver State,Receiver Zip,Receiver Country,Entry Date,Entry Number,Customs Value,Customs Value Currency,Duty & Taxes,Bill Amount,id,Map,Freight,Fuel,Residential,Surcharge,Freight Adj,Fuel Adj,Surcharge Adj,Duties/Tax
4552,33370,2025-03-11,582545100,4623466,Wholesale,NaN,UPS,45715,1Z83079W0313201492,Ground Commercial,TRANSPORTATION,1.0,PKG,36.1,37.0,2.0,2.0,2.0,7,CA,US,NaN,CVS None,2800 ENTERPRISE ST,NaN,INDIANAPOLIS,IN,462191103,US,NaN,NaN,0.0,NaN,0.0,22.14,4553,Freight,22.14,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4553,33370,2025-03-11,582545100,4623466,Wholesale,NaN,UPS,45715,1Z83079W0313201492,Ground Commercial,FUEL SURCHARGE,1.0,PKG,36.1,37.0,2.0,2.0,2.0,7,CA,US,NaN,CVS None,2800 ENTERPRISE ST,NaN,INDIANAPOLIS,IN,462191103,US,NaN,NaN,0.0,NaN,0.0,2.79,4554,Fuel,0.00,2.79,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
invoice_charge_df.columns

Index(['Invoice', 'Invoice Date', 'Order ID', 'Order Number', 'Profile',
       'Reference', 'Carrier', 'Transaction Date', 'Tracking Number',
       'Service Type', 'Charge', 'Package Quantity', 'Packaging Type',
       'Entered Weight (LB)', 'Billed Weight (LB)', 'Dim Length', 'Dim Width',
       'Dim Height', 'Zone', 'Origin State', 'Origin Country', 'Receiver Name',
       'Receiver Company', 'Receiver Address Line 1',
       'Receiver Address Line 2', 'Receiver City', 'Receiver State',
       'Receiver Zip', 'Receiver Country', 'Entry Date', 'Entry Number',
       'Customs Value', 'Customs Value Currency', 'Duty & Taxes',
       'Bill Amount', 'id', 'Map', 'Freight', 'Fuel', 'Residential',
       'Surcharge', 'Freight Adj', 'Fuel Adj', 'Surcharge Adj', 'Duties/Tax'],
      dtype='object')

In [11]:
invoice_Flatten.columns

Index(['Invoice', 'Invoice Date', 'Order ID', 'Order Number', 'Profile',
       'Reference', 'Carrier', 'Transaction Date', 'Tracking Number',
       'Service Type', 'Charge', 'Package Quantity', 'Packaging Type',
       'Entered Weight (LB)', 'Billed Weight (LB)', 'Dim Length', 'Dim Width',
       'Dim Height', 'Zone', 'Origin State', 'Origin Country', 'Receiver Name',
       'Receiver Company', 'Receiver Address Line 1',
       'Receiver Address Line 2', 'Receiver City', 'Receiver State',
       'Receiver Zip', 'Receiver Country', 'Entry Date', 'Entry Number',
       'Customs Value', 'Customs Value Currency', 'Duty & Taxes',
       'Bill Amount', 'id', 'Map', 'Freight', 'Fuel', 'Residential',
       'Surcharge', 'Freight Adj', 'Fuel Adj', 'Surcharge Adj', 'Duties/Tax'],
      dtype='object')